In [7]:
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.decomposition import LatentDirichletAllocation
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
import time
import math

# 엑셀 파일에서 데이터 로드
data = pd.read_excel("잡플래닛_모든_회사_리뷰_총정리.xlsx")

# step6. 각 직무에 대한 토픽을 분석하기 위해 LDA 함수 정의
def perform_topic_modeling(data, n_topics=5, n_words=10):
    # CountVectorizer를 통해 단어 빈도수 행렬 생성
    vectorizer = CountVectorizer(min_df=1, stop_words='english')  # max_df를 제거
    data_vectorized = vectorizer.fit_transform(data['장점'] + " " + data['단점'] + " " + data['경영진에게 바라는 점'])

    # LDA 모델 학습
    lda = LatentDirichletAllocation(n_components=n_topics, random_state=0)
    lda.fit(data_vectorized)

    # 토픽별 주요 단어 추출
    words = vectorizer.get_feature_names_out()
    topics = {}
    for idx, topic in enumerate(lda.components_):
        topic_words = [words[i] for i in topic.argsort()[-n_words:]]
        topics[f"토픽 {idx+1}"] = topic_words
    
    return topics

# 직무별 토픽 모델링 실행 함수
def analyze_topics_by_job(data):
    job_topics = {}
    for job in data['직무'].unique():
        job_data = data[data['직무'] == job]
        if not job_data.empty:
            print(f"{job} 직무의 주요 토픽 분석 중...")
            job_topics[job] = perform_topic_modeling(job_data, n_topics=3)

    # 직무별 주요 토픽 출력
    for job, topics in job_topics.items():
        print(f"\n{job} 직무의 주요 토픽:")
        for topic, words in topics.items():
            print(f"{topic}: {', '.join(words)}")

# 분석 시작
analyze_topics_by_job(data)


개발 직무의 주요 토픽 분석 중...
전문직 직무의 주요 토픽 분석 중...
연구개발 직무의 주요 토픽 분석 중...
인사/총무 직무의 주요 토픽 분석 중...
미디어/홍보 직무의 주요 토픽 분석 중...
엔지니어링 직무의 주요 토픽 분석 중...
생산/제조 직무의 주요 토픽 분석 중...
마케팅/시장조사 직무의 주요 토픽 분석 중...
서비스/고객지원 직무의 주요 토픽 분석 중...

개발 직무의 주요 토픽:
토픽 1: 짧은, 진행하기에, 고민이, 급여, 만족스럽고, 것이, 인력, 있음, 구조, 사람이
토픽 2: 되고, 전달을, 비교하면, 자랑스러워요, 좋지만, 들어서, 복지, 같습니다, 좋을, 많은
토픽 3: 조직, 만들어주세요, 좋게, 만들어서, 때문에, 있어요, 실력을, 의견, 있는, 복지

전문직 직무의 주요 토픽:
토픽 1: 분위기는, 받으며, 바라는, 몰두할, 때가, 동료, 능력있는, 눈치보지, 성장할, 함께
토픽 2: 분위기는, 받으며, 바라는, 몰두할, 때가, 동료, 능력있는, 눈치보지, 성장할, 함께
토픽 3: 받으며, 바라는, 몰두할, 때가, 동료, 능력있는, 눈치보지, 성장할, 함께, 있는

연구개발 직무의 주요 토픽:
토픽 1: 성과급, 사업이, 사업을, 주는지는, 주변에는, 중순이, 사람은, 선택해서, 싸이클에, 광파는
토픽 2: 있고, 있다, 있도록, 동일하니, 스트레스, 동료들이, 괜찮음, 문화가, 기업, 책임감
토픽 3: 사이드잡이, 부서에, 미래를, 정치질만, 예측하는, 좋아하지, 많음, 않음, 없습니다, 말고

인사/총무 직무의 주요 토픽:
토픽 1: 반복으로, 우수했음, 모습을, 의사결정의, 매우, 말아야하지만, 있었으며, 무엇인지, 식당, 사내
토픽 2: 우수했음, 모습을, 의사결정의, 매우, 말아야하지만, 있었으며, 무엇인지, 좋은, 식당, 사내
토픽 3: 복학, 파견직으로, 분들이, 대기업, 식대, 위해, 외에는, 휴학생, 좋은, 생각합니다

미디어/홍보 직무의 주요 토픽:
토픽 1: 사옥이, 사람들도, 발전, 문화가,

In [8]:
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.decomposition import LatentDirichletAllocation
import re

# 불용어 리스트 정의
stop_words = ['있고', '있음', '되고', '하는', '하기', '있습니다', '때문에', '것이', '많은', '같습니다', '하는데', '없는', '다른', '들어', '자주', '때문', '되기']

# 불용어를 제거하고, 불필요한 문자나 공백을 제거하는 전처리 함수
def preprocess_text(text):
    # 모든 불용어 제거
    text = ' '.join(word for word in text.split() if word not in stop_words)
    # 특수문자 제거 및 필요없는 공백 제거
    text = re.sub(r'\W+', ' ', text)
    return text

# 전처리된 텍스트로 LDA 함수 수정
def perform_topic_modeling(data, n_topics=5, n_words=10):
    # 불용어 제거 전처리
    data['processed_text'] = (data['장점'] + " " + data['단점'] + " " + data['경영진에게 바라는 점']).apply(preprocess_text)
    
    # CountVectorizer를 통해 단어 빈도수 행렬 생성
    vectorizer = CountVectorizer(min_df=1)
    data_vectorized = vectorizer.fit_transform(data['processed_text'])

    # LDA 모델 학습
    lda = LatentDirichletAllocation(n_components=n_topics, random_state=0)
    lda.fit(data_vectorized)

    # 토픽별 주요 단어 추출
    words = vectorizer.get_feature_names_out()
    topics = {}
    for idx, topic in enumerate(lda.components_):
        topic_words = [words[i] for i in topic.argsort()[-n_words:]]
        topics[f"토픽 {idx+1}"] = topic_words
    
    return topics

# 직무별 토픽 모델링 실행 함수
def analyze_topics_by_job(data):
    job_topics = {}
    for job in data['직무'].unique():
        job_data = data[data['직무'] == job]
        if not job_data.empty:
            print(f"{job} 직무의 주요 토픽 분석 중...")
            job_topics[job] = perform_topic_modeling(job_data, n_topics=3)

    # 직무별 주요 토픽 출력
    for job, topics in job_topics.items():
        print(f"\n{job} 직무의 주요 토픽:")
        for topic, words in topics.items():
            print(f"{topic}: {', '.join(words)}")

# 엑셀 파일에서 데이터 로드
data = pd.read_excel("잡플래닛_모든_회사_리뷰_총정리.xlsx")
# 분석 시작
analyze_topics_by_job(data)


개발 직무의 주요 토픽 분석 중...
전문직 직무의 주요 토픽 분석 중...
연구개발 직무의 주요 토픽 분석 중...
인사/총무 직무의 주요 토픽 분석 중...
미디어/홍보 직무의 주요 토픽 분석 중...
엔지니어링 직무의 주요 토픽 분석 중...
생산/제조 직무의 주요 토픽 분석 중...
마케팅/시장조사 직무의 주요 토픽 분석 중...
서비스/고객지원 직무의 주요 토픽 분석 중...

개발 직무의 주요 토픽:
토픽 1: 조직, 좋게, 변화가, 업무, 문화가, 지키기, 장기적으로, it, 복지, 의견
토픽 2: 연구, 예전처럼, 이외에도, 있다, 있으면, 자유롭게, 맛있는, 힘들, 있는, 좋을
토픽 3: 개선, 고민이, 만족스럽고, 수준이, 보임, 필요, 인력, 복지, 구조, 사람이

전문직 직무의 주요 토픽:
토픽 1: 분위기는, 받으며, 바라는, 몰두할, 때가, 동료, 능력있는, 눈치보지, 성장할, 함께
토픽 2: 분위기는, 받으며, 바라는, 몰두할, 때가, 동료, 능력있는, 눈치보지, 성장할, 함께
토픽 3: 받으며, 바라는, 몰두할, 때가, 동료, 능력있는, 눈치보지, 성장할, 함께, 있는

연구개발 직무의 주요 토픽:
토픽 1: 모르겠음, 올해, 위한, 따른, 사람들, 자체의, 아닌, 일하고, 없습니다, 광파는
토픽 2: 있게, 있다, 서로, 뛰어난, 동일하니, 동료들이, 말고, 문화가, 책임감, 기업
토픽 3: 부서의, 솔직히, 않은데, 야근, 연구원인지, 연봉, 워라벨좀, 챙겨주셈, cs업체, 괜찮음

인사/총무 직무의 주요 토픽:
토픽 1: 반복으로, 우수했음, 모습을, 의사결정의, 매우, 말아야하지만, 있었으며, 무엇인지, 식당, 사내
토픽 2: 우수했음, 모습을, 의사결정의, 매우, 말아야하지만, 있었으며, 무엇인지, 좋은, 식당, 사내
토픽 3: 복학, 파견직으로, 분들이, 대기업, 식대, 위해, 외에는, 휴학생, 좋은, 생각합니다

미디어/홍보 직무의 주요 토픽:
토픽 1: 사옥이, 사람들도, 발전, 문화가, 멀고, 단점, 너무, 고안해야

C:\Users\jhpar\AppData\Local\Temp\ipykernel_21156\567886034.py:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['processed_text'] = (data['장점'] + " " + data['단점'] + " " + data['경영진에게 바라는 점']).apply(preprocess_text)
C:\Users\jhpar\AppData\Local\Temp\ipykernel_21156\567886034.py:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['processed_text'] = (data['장점'] + " " + data['단점'] + " " + data['경영진에게 바라는 점']).apply(preprocess_text)
C:\Users\jhpar\AppData\Local\Temp\ipykernel_21156\567886034.py:20: Set